<a href="https://colab.research.google.com/github/srishtimishra2/music-recommendation-system-based-on-facial-emotion-recognition/blob/main/Data_Preparation_with_Mood_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
file= files.upload()
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt


!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

Saving playlists.csv to playlists.csv


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [28]:
from google.colab import files
file= files.upload()





Saving genres_v2.csv to genres_v2 (3).csv


In [29]:
spotify_df = pd.read_csv('genres_v2 (3).csv')

In [30]:
spotify_df.isna().sum()


danceability            0
energy                  0
key                     0
loudness                0
mode                    0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
type                    0
id                      0
uri                     0
track_href              0
analysis_url            0
duration_ms             0
time_signature          0
genre                   0
song_name           20786
Unnamed: 0          21525
title               21525
dtype: int64

In [31]:
spotify_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,audio_features,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,audio_features,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,audio_features,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,audio_features,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,audio_features,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [32]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [33]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [34]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [35]:
mood_trans_np[10011]

array([ 2.33887e+05,  9.08000e-01,  1.49000e-02,  5.67000e-01,
        7.04000e-06,  7.49000e-02,  2.55000e-01, -6.18100e+00,
        2.85000e-01,  1.36016e+02])

In [38]:
from google.colab import files
file= files.upload()

Saving data_moods.csv to data_moods.csv


In [39]:
df = pd.read_csv('data_moods.csv')

In [40]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [41]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


In [42]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [43]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [44]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: 76.24% (5.27%)


In [49]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

In [50]:
y_preds

array([1, 0, 0, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 3, 3, 1, 3, 3, 0, 0,
       3, 0, 1, 3, 0, 3, 3, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 3, 1, 0, 0, 3,
       3, 3, 0, 2, 1, 0, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 0, 2, 0, 3, 1,
       2, 2, 0, 3, 1, 0, 3, 0, 0, 0, 3, 3, 1, 0, 3, 1, 1, 1, 0, 1, 3, 0,
       0, 1, 1, 0, 1, 3, 3, 1, 1, 0, 3, 1, 2, 0, 0, 3, 3, 2, 2, 0, 1, 1,
       3, 3, 0, 0, 2, 1, 3, 3, 3, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1, 3, 3,
       1, 1, 1, 3, 3, 1])

In [51]:
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f69acb56610>)])

In [52]:
def predict_mood(preds):

    preds_features = np.array(preds[:]).reshape(-1,1).T
    results = pip.predict(preds_features)
    mood = np.array(target['mood'][target['encode']==int(results)])
    return str(mood[0])

In [53]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

In [54]:
spotify_df.shape

(42305, 22)

In [55]:
print(len(res))

42305


In [56]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [57]:
spotify_df.to_csv('kaggleMusicMoodFinal.csv')

In [58]:
res.count("Sad")

10358

In [59]:
res.count("Happy")

4739

In [60]:
res.count("Energetic")

26412

In [61]:
res.count("Calm")

796

In [62]:
spotify_df.shape

(42305, 23)